In [ ]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--conf spark.cassandra.connection.host=cassandra --packages org.apache.spark:spark-streaming-kafka-0-8_2.11:2.0.2,com.datastax.spark:spark-cassandra-connector_2.11:2.0.2 pyspark-shell'

In [ ]:
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
from pyspark.streaming.kafka import KafkaUtils
from pyspark.sql import SQLContext
from pyspark.sql import functions as F
from pyspark.sql.types import *

In [ ]:
sc = SparkContext(appName="BigDataRiver")
sc.setLogLevel("WARN")
sc.setCheckpointDir('checkpoint/')
ssc = StreamingContext(sc, 1)
sql = SQLContext(sc)

In [ ]:
kafkaStream = KafkaUtils.createDirectStream(ssc, ['bdr'], {"metadata.broker.list": 'kafka:9092'})

In [ ]:
parsed = kafkaStream.map(lambda v: v[1])

In [ ]:
schema = StructType([   
    StructField("purchased_count", LongType(), False),
    StructField("clicked_count", LongType(), False)
])

def separateClicks(is_purchase):
  return (is_purchase, 1-is_purchase)

separateClicks_udf = F.udf(separateClicks, schema)

In [ ]:
def bdr(rdd):
    df0 = sql.read.json(rdd)
    if(len(df0.columns)):
        df0.withColumn('c', separateClicks_udf(df0['is_purchase'])).select("user_id","product",F.col("product_generic_category").alias("generic_cat"), F.col("product_specific_category").alias("specific_cat"),"c.purchased_count","c.clicked_count").write.format("org.apache.spark.sql.cassandra").mode('append').options(table="users_interests", keyspace="bdr").save()
        print "Saved"
    else:
        print "Empty"
    

In [ ]:
parsed.foreachRDD(lambda rdd: bdr(rdd))

In [ ]:
ssc.start()
ssc.awaitTermination()